In [4]:
import pickle
import numpy as np
from scipy.spatial.distance import cdist

def get_fire_stats(layer_feat_b):
    stats1 = []
    stats2 = []
    for nn in range(len(layer_feat_b)):
        stats1.append(np.mean(np.sum(layer_feat_b[nn], axis=2)))
        stats2.append(np.sum(np.sum(layer_feat_b[nn], axis=2)==0)/(layer_feat_b[nn].shape[0]*layer_feat_b[nn].shape[1]))
        
    return np.mean(stats1), np.mean(stats2)


def get_IOU(layer_feat_b1, layer_feat_b2):
    stats1 = []
    stats2 = []
    stats3 = []
    for nn in range(len(layer_feat_b1)):
        intr = np.sum(np.logical_and(layer_feat_b1[nn], layer_feat_b2[nn]))
        union = np.sum(np.logical_or(layer_feat_b1[nn], layer_feat_b2[nn]))
        stats1.append(intr/union)
        stats2.append(intr/np.sum(layer_feat_b1[nn]))
        stats3.append(intr/np.sum(layer_feat_b2[nn]))
        
    return(np.mean(stats1), np.mean(stats2), np.mean(stats3))


def get_detail(layer_feat_b1, layer_feat_b2):
    stats1 = []
    stats2 = []
    zero_cnt = 0
    single_cnt = 0
    for nn in range(len(layer_feat_b1)):
        rr, cc, vch = np.where(layer_feat_b1[nn] != layer_feat_b2[nn])
        for ri, ci in zip(rr, cc):
            stats1.append(np.sum(layer_feat_b1[nn][ri, ci]))
            stats2.append(np.sum(layer_feat_b2[nn][ri, ci]))
            if np.sum(layer_feat_b1[nn][ri, ci]) == 0:
                zero_cnt += 1
                
            if np.sum(layer_feat_b2[nn][ri, ci]) == 0:
                single_cnt += 1
            
                
    return(np.mean(stats1), np.mean(stats2), zero_cnt, single_cnt)

In [14]:
category = 'car'
target = 'bus'
magic_thh = 0.47

adv_feat_file = '/export/home/qliu24/VC_adv_data/qing/VGG_adv/feat/pool4FeatVC_adv_{}_{}.pickle'.format(category, target)
with open(adv_feat_file, 'rb') as fh:
    layer_feature, layer_feature_fool = pickle.load(fh)
    
dict_file='/export/home/qliu24/qing_voting_139/qing_voting_py/data/dictionary_PASCAL3D+_all_VGG16_pool4_K200_vMFMM30.pickle'

with open(dict_file, 'rb') as fh:
    _, centers, _ = pickle.load(fh)

N = len(layer_feature)
r_set = [None for nn in range(N)]
for nn in range(N):
    iheight,iwidth = layer_feature[nn].shape[0:2]
    lff = layer_feature[nn].reshape(-1, 512)
    lff_norm = lff/np.sqrt(np.sum(lff**2, 1)).reshape(-1,1)
    r_set[nn] = cdist(lff_norm, centers, 'cosine').reshape(iheight,iwidth,-1)
    
layer_feature_b = [None for nn in range(N)]
for nn in range(N):
    layer_feature_b[nn] = (r_set[nn]<magic_thh).astype(int)
    
    
r_set_fool = [None for nn in range(N)]
for nn in range(N):
    iheight,iwidth = layer_feature_fool[nn].shape[0:2]
    lff = layer_feature_fool[nn].reshape(-1, 512)
    lff_norm = lff/np.sqrt(np.sum(lff**2, 1)).reshape(-1,1)
    r_set_fool[nn] = cdist(lff_norm, centers, 'cosine').reshape(iheight,iwidth,-1)
    
layer_feature_b_fool = [None for nn in range(N)]
for nn in range(N):
    layer_feature_b_fool[nn] = (r_set_fool[nn]<magic_thh).astype(int)
    
print('basic per-pixel-firing and coverage:')
print(get_fire_stats(layer_feature_b))
print(get_fire_stats(layer_feature_b_fool))  

print('IOU:')
print(get_IOU(layer_feature_b,layer_feature_b_fool))


a,b,c,d = get_detail(layer_feature_b,layer_feature_b_fool)
total_pixel = np.sum([np.prod(lfb.shape[0:2]) for lfb in layer_feature_b])
total_diff_pixel = np.sum([np.sum(np.sum(layer_feature_b[nn]!=layer_feature_b_fool[nn], axis=2)!=0) for nn in range(len(layer_feature_b))])
print('diff/total pixel: {}/{} ({})'.format(total_diff_pixel, total_pixel, total_diff_pixel/total_pixel))
print('details in diff pixel: {}, {}'.format(c/total_diff_pixel, d/total_diff_pixel))

basic per-pixel-firing and coverage:
(2.0722921267927354, 0.23907610497185605)
(1.9321953762227246, 0.2686864945762546)
IOU:
(0.69101815863622884, 0.78911390017793759, 0.84415350857147819)
diff/total pixel: 157496/392756 (0.4010021489168848)
details in diff pixel: 0.12259993904607101, 0.27416569309696753


In [10]:
59749/total_pixel

0.1521275295603377